In [ ]:
# The Awesome Python Package for many ML functions
import turicreate as tc

movies = tc.SFrame.read_csv("Movies_and_TV.csv")

movies.show()

In [ ]:
#Creating a Item-Item CF Model
model = tc.recommender.item_similarity_recommender.create(movies, user_id='user', item_id='item', target='rating', similarity_type = 'pearson')

In [ ]:
#Code for Calculating the NDCG Metric
import numpy as np


def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [ ]:
#Creating a DataFrame from the dataset
df = pd.read_csv("Movies_and_TV.csv", sep=',', header=None)
df.columns = ['item', 'user', 'rating']

columns_titles = ["user","item", "rating"]
                    
df=df.reindex(columns=columns_titles)

In [ ]:
# This is the pipeline used for mining the required data
import csv
import pandas as pd
import turicreate as tc

csvfile = open('results.csv', 'w')
resultwriter = csv.writer(csvfile)
resultwriter.writerow(['User', 'Original Items', 'Num of Recs(K_O)', 'Rating', 'Num_of_preds' 'Hits', 'Precision'])
csvfile.close()

count = 0
try:
    users = ['A6ADO7B6FUVN', 'AYWSFRCIMOAYE', 'A2R6RA8FRBS608', 'A25ZVI6RH1KA5L', 'A13E0ARAXI6KJW', 'A2ZB8B7VQONZA6', 'A2FPDWTD9AENVK', 'A1TA5QYECZP1L1', 'A10H47FMW8NHII', 'ANAYSRE3LX8GZ']
    ratings  = [3.5, 5]
    for user in users:
        original_items = df.loc[df['user'] == user]['item'].tolist()
        original_rating = df.loc[df['user'] == user]['rating'].tolist()
        n = len(original_items)
        
        percentages = [0]
        #, 25, 50, 75, 100
        
        for percentage in percentages:
            c = (percentage/100) * n
            partial_items = original_items[:int(c)]
            partial_rating = original_rating[:int(c)]
            
            remaining_items = [i for i in original_items if i not in partial_items]
            remaining_ratings = []
            
            for item in remaining_items:
                remaining_ratings.append(original_rating[original_items.index(item)])
                
            y = len(partial_items)

            n = n-y
            recs = range(500, 0, -50)
            for t in recs:
                a = model.recommend([user], t)

                for rating in ratings:
                    nf = df[df.user != user]
                    for item in a['item']:
                        nf = nf.append({'item' : item , 'user' : user , 'rating': rating} , ignore_index=True)
                        
                    for i, item in enumerate(partial_items):
                        nf = nf.append({'item' : item , 'user' : user , 'rating': partial_rating[i]} , ignore_index=True)

                    nframe = tc.SFrame(nf)
                    model2 = tc.recommender.item_similarity_recommender.create(nframe, user_id='user', item_id='item', target='rating', similarity_type = 'pearson')
                    
                    for k_a in range(1, 501, 50):
                        recomm_items = model2.recommend([user], k_a)['item']


                        result = [item for item in recomm_items if item in remaining_items]
                        relevance = []
                        for item in recomm_items:
                            if item in remaining_items:
                                relevance.append(remaining_ratings[remaining_items.index(item)])
                            else:
                                relevance.append(0)




                        if len(result) == 0:
                            with open("results.csv", "a+") as csvfile:
                                resultwriter = csv.writer(csvfile)
                                resultwriter.writerow([user, len(original_items), t, rating, len(recomm_items), 0, 0])
                        else:
                            precision = len(result) / n
                            print("Result: ", [user, len(original_items), t, rating, len(recomm_items), len(result), precision])
                            with open("results.csv", "a+") as csvfile:
                                resultwriter = csv.writer(csvfile)
                                resultwriter.writerow([user, len(original_items), t, rating, len(recomm_items), len(result), precision])
except Exception as e:
    print(e)

In [ ]:
# This code snippet is responsible for generating the required graphs
import matplotlib
import csv
import pandas as pd
# values = []

rating_three = []
rating_five = []

with open("rf_no_hist_results.csv", "r") as csvfile:
    reader2 = csv.DictReader(csvfile)
    for row in reader2:
        flag = False
        if row['Rating'] == '3.5':
            
            for entry in rating_three:
                if row['Num of Recs'] == entry['x'] and row['Recomms_Observed'] == entry['y']:
                    entry['Precision'].append(float(row['Precision']))
                    flag = True
                    break
            if flag:
                continue

            temp = {}
            temp['x'] = row['Num of Recs']
            temp['y'] = row['Recomms_Observed'] 
            temp['Precision'] = [] 
            temp['Precision'].append(float(row['Precision']))
            rating_three.append(temp)
        
        elif row['Rating'] == '5':
            
            for entry in rating_five:
                if row['Num of Recs'] == entry['x'] and row['Recomms_Observed'] == entry['y']:
                    entry['Precision'].append(float(row['Precision']))
                    flag = True
                    break
            if flag:
                continue

            temp = {}
            temp['x'] = row['Num of Recs']
            temp['y'] = row['Recomms_Observed'] 
            temp['Precision'] = [] 
            temp['Precision'].append(float(row['Precision']))
            rating_five.append(temp)
            
            
# print(rating_three[0])

for temp in rating_five:
    my_list = temp['Precision']
    my_avg = sum(my_list)/len(my_list)
    temp['Precision'] = float(my_avg)

# print(rating_three[0]) 

five = []

for temp in rating_five:
    p = []
    p.append((temp['x']))
    p.append((temp['y']))
    p.append(float(temp['Precision']))
    
    five.append(p)
    
pf = pd.DataFrame(five, columns=['Predictions', 'Recomms_Observed', 'Avg_Precision'])

# print(pf)


ax1 = pf.plot.scatter(x='Predictions', y='Recomms_Observed', c='Avg_Precision')
ax1.set_xlabel('Predictions')
ax1.figure.savefig('rf_no_hist_5.0.png')
# print(three)